In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import re
import scipy.optimize as opt
from tqdm import tqdm
import sympy as sp
from sympy.parsing.sympy_parser import parse_expr
from sympy import lambdify

In [3]:
#filename = 'pareto_low_adv.csv'
filename = 'pareto_low_sim.csv'

In [6]:
t_eq=pd.read_csv('/data/zj448/SR/Ultimate_paper/pareto_archive/'+filename)
len(t_eq)

2105

In [5]:
# drop rows that have the same value in column sympy_format.
t_eq=t_eq.drop_duplicates(subset='sympy_format',keep='first')
len(t_eq)

1341

In [6]:
t_eq['fitting_format']=''
t_eq['num_fitting_variables']=0
t_eq['initial_constant_guess'] = pd.Series(dtype='object')

number_matching_pattern = r"(?<![a-zA-Z0-9_.])[+-]?(\d+\.\d+|\.\d+|\d+\.|\d+)(?:[eE][-+]?\d+)?"

for row in tqdm(t_eq.iterrows(),total=len(t_eq)):
    #equation = row[1]['sympy_format']
    #equation = parse_expr(equation)

    raw_equation = row[1]['equation']
    # replace the power sign
    raw_equation = raw_equation.replace('^','**')
    equation = str(parse_expr(raw_equation))

    # # swap constants
    # constants = re.findall(number_matching_pattern, equation)


    # variable_list=[]
    # for i in range(len(constants)):
    #     variable_list.append('p['+str(i)+']')


    # for i in range(len(constants)):
    #     equation = equation.replace(constants[i],variable_list[i])

    # swap constants v2
    # Find all unique matches first to avoid duplicates and incorrect indexing
    constants = list(set(re.findall(number_matching_pattern, equation)))
    # Sort constants by their length in descending order to replace longer numbers first, preventing partial replacement issues
    constants.sort(key=len, reverse=True)

    def replace_with_variable(match):
        # Find the matched number in the constants list and get its index
        number = match.group(1)
        index = constants.index(number)

        # Check if the match includes a minus sign
        if match.group(0).startswith('-'):
            sign = '-'
        else:
            sign = ''
        return f'{sign}p[{index}]'

    equation = re.sub(number_matching_pattern, replace_with_variable, equation)

    t_eq.loc[row[0],'fitting_format']=str(equation) 
    t_eq.loc[row[0],'num_fitting_variables']=len(constants)
    t_eq.at[row[0],'initial_constant_guess']=constants

  0%|          | 0/1341 [00:00<?, ?it/s]

100%|██████████| 1341/1341 [00:05<00:00, 227.33it/s]


In [7]:
t_eq=t_eq.drop_duplicates(subset='fitting_format',keep='last')
len(t_eq)

1042

In [8]:
#t_eq.to_csv('/data/zj448/SR/Ultimate_paper/pareto_archive/pareto_cleaned.csv',index=False)
t_eq.to_csv('/data/zj448/SR/Ultimate_paper/pareto_archive/'+filename[:-4]+'_cleaned.csv',index=False)

In [9]:
t_eq

,complexity,loss,score,equation,sympy_format,lambda_format,number_constants,variables,number_variables,unique_number_variables,evolutions,iterations,fitting_format,num_fitting_variables,initial_constant_guess
0,1,2.422899,0.000000,x66,x66,PySRFunction(X=>x66),0,{'x66'},1,1,0,0,x66,0,[]
1,3,0.793023,0.558434,(x73 - -8.377161567514106),x73 + 8.377161567514106,PySRFunction(X=>x73 + 8.377161567514106),1,{'x73'},1,1,0,0,x73 + p[0],1,[8.377161567514106]
8,17,0.090620,0.001319,((((0.13631070516777483 - ((x42 * (0.556125855...,-0.46524650939821102*x42 + x53 + x69 + 0.39610...,PySRFunction(X=>-0.46524650939821102*x42 + x53...,6,"{'x53', 'x69', 'x42'}",3,3,0,0,-p[1]*x42 + x53 + x69 + p[0],2,"[0.39610601048669413, 0.46524650939821102]"
9,19,0.090473,0.000813,((((0.45731312294876897 - ((x42 * 0.5561258553...,-0.45018841361960323*x42 + x53 + x69 + 0.45731...,PySRFunction(X=>-0.45018841361960323*x42 + x53...,6,"{'x53', 'x69', 'x42', 'x51'}",4,4,0,0,-p[2]*x42 + x53 + x69 + p[1] - p[4]/(p[3]*x51 ...,5,"[0.99345268417633044, 0.45731312294876897, 0.4..."
10,20,0.084512,0.068159,(((exp(exp((1.6009789062395106 - x52) ^ -0.446...,1.1130255347463974*exp(exp(0.81048234765746345...,PySRFunction(X=>1.1130255347463974*exp(exp(0.8...,6,"{'x23', 'x52', 'x43'}",3,3,0,0,p[4]*exp(exp(p[0]/(p[6] - p[1]*x52)**p[5])) + ...,7,"[0.81048234765746345, 0.62461784855671134, 0.7..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
342596,23,0.011585,0.056926,((((x15 / 0.31852133710431946) + (exp(x79) ^ -...,3.1395071020704912*x15 + (x39 - x42)/(x17/(x28...,PySRFunction(X=>3.1395071020704912*x15 + (x39 ...,3,"{'x42', 'x79', 'x19', 'x39', 'x15', 'x64', 'x1...",8,8,4,4495,p[1]*x15 + (x39 - x42)/(x17/(x28*x64) + exp(x1...,3,"[0.0003954498187632686, 3.1395071020704912, 0...."
342627,22,0.012256,0.010969,((((x15 / 0.31852133710431946) + (x23 ^ -0.000...,3.1395071020704912*x15 + x23**(-0.000395449818...,PySRFunction(X=>3.1395071020704912*x15 + x23**...,4,"{'x42', 'x19', 'x23', 'x39', 'x15', 'x64', 'x28'}",7,7,4,4497,p[2]*x15 + x23**(-p[0]) + (x39 - x42)/(exp(x19...,4,"[0.0003954498187632686, 0.17239071180936974, 3..."
343962,6,0.044143,0.436999,(log10(x15) * (26.037329703202072 - x42)),(26.037329703202072 - x42)*log(x15)/log(10),PySRFunction(X=>(26.037329703202072 - x42)*log...,1,"{'x42', 'x15'}",2,2,4,4581,(p[0] - x42)*log10(x15),1,[26.037329703202072]
344783,12,0.014401,0.510440,(((x15 / 0.31852133710431946) + ((x39 - x42) /...,3.1395071020704912*x15 + (x39 - x42)*exp(-x19)...,PySRFunction(X=>3.1395071020704912*x15 + (x39 ...,2,"{'x19', 'x42', 'x15', 'x39'}",4,4,4,4632,p[1]*x15 + (x39 - x42)*exp(-x19) + p[0],2,"[0.6625382197895642, 3.1395071020704912]"


In [12]:
t_eq.iloc[5]

complexity                                                                22
loss                                                                0.082313
score                                                               0.013183
equation                   (((exp(exp((1.6009789062395106 - x52) ^ -0.446...
sympy_format               1.1130255347463974*exp(exp(0.81048234765746345...
lambda_format              PySRFunction(X=>1.1130255347463974*exp(exp(0.8...
number_constants                                                           6
variables                                        {'x23', 'x52', 'x43', 'x3'}
number_variables                                                           4
unique_number_variables                                                    4
evolutions                                                                 0
iterations                                                                 0
fitting_format             p[4]*exp(exp(p[0]/(p[6] - p[1]*x52)**p[5])) + ...